In [3]:
from tkinter import *
from tkinter import filedialog
from PIL import Image
from PIL import ImageTk
import cv2
import imutils
import numpy as np

In [4]:
from matplotlib import pyplot as plt

In [10]:
def calculo():
    global image
    global imageToShowOutput
    global bordes, colorBorde
    global valormin, valormax
    global lblInfo4
   
    kernel = np.ones((3,3), np.uint8)
    img2 = image.copy()
    
    ## Convertimos a escala de grises  
    imageGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Aplicar suavizado Gaussiano
    gauss = cv2.GaussianBlur(imageGray, (9,9), 0)
    
    
    if umbralMin.get() == 0.0:
        valormin = 100
    if umbralMax.get() == 0.0:
        valormax = 255
        
    _, th = cv2.threshold(imageGray, valormin, valormax, cv2.THRESH_BINARY)
    
    
    #_, th = cv2.threshold(imageGray, 100, 255, cv2.THRESH_BINARY)
    #Aplicamos tecnica Morfologica de Cierre para eliminar ruido
    closing = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
    
    # Detectamos los bordes con Canny
    if selected.get() == 1:
        bordes = cv2.Canny(gauss,50,150)
        colorBorde = (0,0,255)
        #plt.imshow(bordes,'gray')
    if selected.get() == 2 or selected.get() == 0:
        bordes = cv2.Canny(closing, 135, 255)
        colorBorde = (0,255,0)
        
    
    contours, jerarquia = cv2.findContours(bordes, cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img2, contours, -1, colorBorde, 3)
    cantidadMonedas = len(contours)
    img2 = cv2.putText(img2, "son "+ str(cantidadMonedas) + " monedas", (10,50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2, cv2.LINE_AA)
    
    print("Son " + str(cantidadMonedas) + " monedas")
    #
    #if selected.get() is null:
    #    selected.get() == 2
    
    if selected.get() == 1:
        imageToShowOutput = gauss
    elif selected.get() == 2:
        imageToShowOutput = closing
    elif selected.get() == 3:
        imageToShowOutput = bordes
    elif selected.get() == 4 or selected.get() == 0:
        imageToShowOutput = img2
        
    
        
    # Para visualizar la imagen en lblOutputImage en la GUI
    im = Image.fromarray(imageToShowOutput)
    img = ImageTk.PhotoImage(image=im)
    lblOutputImage.configure(image=img)
    lblOutputImage.image = img
    # Label IMAGEN DE SALIDA
    lblInfo3 = Label(root, text="IMAGEN DE SALIDA:", font="bold")
    lblInfo3.grid(column=1, row=0, padx=5, pady=5)
    # Label CANTIDAD DE OBJETOS
    #lblInfo4.config(text=' ')
    textAux = "Son " +str(cantidadMonedas) + " monedas"
    lblInfo4 = Label(root, text=textAux, font="bold")
    lblInfo4.grid(column=1, row=9, padx=5, pady=5)

In [6]:
def elegir_imagen():
    # Especificar los tipos de archivos, para elegir solo a las imágenes
    path_image = filedialog.askopenfilename(filetypes = [
        ("image", ".jpeg"),
        ("image", ".png"),
        ("image", ".jpg"),
        ("image",".bmp")])

    if len(path_image) > 0:
        global image
        # Leer la imagen de entrada y la redimensionamos
        image = cv2.imread(path_image)
        image= imutils.resize(image, height=380)
        # Para visualizar la imagen de entrada en la GUI
        imageToShow= imutils.resize(image, width=180)
        imageToShow = cv2.cvtColor(imageToShow, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(imageToShow )
        img = ImageTk.PhotoImage(image=im)
        lblInputImage.configure(image=img)
        lblInputImage.image = img
        # Label IMAGEN DE ENTRADA
        lblInfo1 = Label(root, text="IMAGEN DE ENTRADA:")
        lblInfo1.grid(column=0, row=1, padx=5, pady=5)
        # Al momento que leemos la imagen de entrada, vaciamos
        # la iamgen de salida y se limpia la selección de los
        # radiobutton
        lblOutputImage.image = ""
        selected.set(0)


In [7]:
def guardar_imagen():
    #global imageOutput
    #global image
    global imageToShowOutput
    #aux = combo.get() 
    aux='IMG'
    im = Image.fromarray(imageToShowOutput.astype(np.uint8))
    #im = Image.fromarray((image * 255).astype(np.uint8))
    im.save(f"{aux}_imagenOuput.png")
    plt.imshow(im)

In [8]:
# Creamos la ventana principal
root = Tk()

# Label donde se presentará la imagen de entrada
lblInputImage = Label(root)
lblInputImage.grid(column=0, row=2)

# Label donde se presentará la imagen de salida
lblOutputImage = Label(root)
lblOutputImage.grid(column=1, row=1, rowspan=6)

# Creamos el boton para realizar el calculo( closing, bordes, resultado)
btnCalculo = Button(root, text="Calcular", width=25, command=calculo)
btnCalculo.grid(column=0, row=3, padx=5, pady=5)

# Label ¿Qué técnica te gustaría resaltar?
lblInfo2 = Label(root, text="¿Qué téctnica te gustaría resaltar?", width=25)
lblInfo2.grid(column=0, row=4, padx=5, pady=5)

# Creamos los radio buttons y la ubicación que estos ocuparán
selected = IntVar()
rad1 = Radiobutton(root, text='Gaussiana', width=25,value=1, variable=selected, command= calculo)
rad2 = Radiobutton(root, text='Closing',width=25, value=2, variable=selected, command= calculo)
rad1.grid(column=0, row=5)
rad2.grid(column=0, row=6)

# Label Bordes dependiendo de la tecnicas
lblInfo3 = Label(root, text="Bordes con Gaussiano/Closing", width=25)
lblInfo3.grid(column=0, row=7, padx=5, pady=5)

#Radio button de Borde
rad3 = Radiobutton(root, text='Bordes',width=25, value=3, variable=selected, command= calculo)
rad3.grid(column=0, row=8)

#Radio button de Resultado
rad4 = Radiobutton(root, text='Resultado',width=25, value=4, variable=selected, command= calculo)
rad4.grid(column=0, row=9)

# Creamos el botón para elegir la imagen de entrada
btn = Button(root, text="Elegir imagen", width=25, command=elegir_imagen)
btn.grid(column=0, row=0, padx=5, pady=5)

# Creamos el botón para salir
btnSalir = Button(root, text="Salir", bg = '#ebaec6', width= 25, command=root.destroy)
btnSalir.grid(column=0, row=10, padx=5, pady=5)

# Label Resultados de cuantos objetos hay
#lblInfo2 = Label(root, text="", font="bold", width=25)
#lblInfo2.grid(column=0, row=8, padx=5, pady=5)

# Creamos el boton para guardar
btnSave = Button(root, text="Guardar imagen", width=25, command=guardar_imagen)
btnSave.grid(column=1, row=10, padx=5, pady=5)
##

#
umbralMin = DoubleVar(value=0.0)
umbralMax = DoubleVar(value=0.0)

#
# Label Resultados de cuantos objetos hay
#lblInfo2 = Label(root, text="", font="bold", width=25)
#lblInfo2.grid(column=0, row=8, padx=5, pady=5)
# label valor a
valor_a_label = Label(root, text="Umbral Min:")
valor_a_label.grid(column=0, row=11, padx=5, pady=10)

valor_a_entry = Entry(root, textvariable=umbralMin, width=20)
valor_a_entry.grid(column=0, row=12, padx=5, pady=10)

# label valor b
valor_b_label = Label(root, text="Umbral Max")
valor_b_label.grid(column=0, row=13, padx=5, pady=10)

valor_b_entry = Entry(root, textvariable=umbralMax, width=20)
valor_b_entry.grid(column=0, row=14, padx=5, pady=10)

root.mainloop()

Son 12 monedas
Son 12 monedas
Son 12 monedas
Son 12 monedas
Son 12 monedas
Son 349 monedas
Son 10 monedas
Son 10 monedas
Son 10 monedas
Son 10 monedas
Son 180 monedas
Son 8 monedas
Son 8 monedas
Son 8 monedas
Son 8 monedas
Son 180 monedas
